In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import pickle


from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from methods import gan, accuracy_XGboost, aae  # , vae
from helperfunctions import *

import xgboost as xgb
import time
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    recall_score,
    precision_score,
    f1_score,
    accuracy_score,
    confusion_matrix,
)

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
file_name = 'data/creditcard.csv'
df = pd.read_csv(file_name)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
df['Amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
df.drop('Time',axis=1, inplace=True)

In [4]:
label_col = []
feature_cols = []
label_col = [i for i in df.columns if 'Class' in i]
feature_cols = [i for i in df.columns if i not in label_col]

In [5]:
X = df.loc[:,feature_cols]
y = df.loc[:,label_col]

In [63]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [64]:
print(f'Train size: {train_X.shape}, Test size: {test_X.shape}')

Train size: (227845, 29), Test size: (56962, 29)


In [66]:
X_sampled,y_sampled = smote.fit_resample(train_X,train_y) #50:50 

C:\Users\chinne\Anaconda3\envs\DGM\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [67]:
train_X_sampled = pd.DataFrame(data=X_sampled, columns=feature_cols)
train_y_sampled = pd.DataFrame(data=y_sampled, columns=label_col)

## XGBoost

In [78]:
import xgboost as xgb
start_time = time.time()
dtrain = xgb.DMatrix(train_X_sampled.values, train_y_sampled.values)
dtest = xgb.DMatrix(test_X.values)
# specify parameters via map
param = {
        'max_depth':5, 
        'eta':0.3, 
        'verbosity': 1, 
        'objective':'binary:logistic', 
        'eval_metric': 'auc', 
        'seed': 42
        }
model = xgb.train(param, dtrain, num_boost_round=100)
# make prediction
preds = model.predict(dtest)
elapsed_time = time.time() - start_time
print('Time elapsed to train: ', elapsed_time)

Time elapsed to train:  56.006611585617065


In [79]:
preds = preds.round()

In [80]:
print('---' * 45)
print('XGBoost ROC/AUC: {:.6f}'.format(roc_auc_score(test_y, preds)))
print('Recall Score: {:.6f}'.format(recall_score(test_y, preds)))
print('Precision Score: {:.6f}'.format(precision_score(test_y, preds)))
print('F1 Score: {:.6f}'.format(f1_score(test_y, preds)))
print('Accuracy Score: {:.6f}'.format(accuracy_score(test_y, preds)))
print('---' * 45)

---------------------------------------------------------------------------------------------------------------------------------------
XGBoost ROC/AUC: 0.929462
Recall Score: 0.867347
Precision Score: 0.150709
F1 Score: 0.256798
Accuracy Score: 0.991363
---------------------------------------------------------------------------------------------------------------------------------------


In [15]:
file_name = 'data/creditcard.csv'
df = pd.read_csv(file_name)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [16]:
df['Amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
df.drop('Time',axis=1, inplace=True)

# Train GAN

In [19]:
df2 = prepareDataset(df, batch_size = 64, device = 'cpu', label = 'no')

In [20]:
df2.name

AttributeError: 'DataLoader' object has no attribute 'name'

In [10]:
xgblosses, G_losses, D_losses = gan.train(df, prefix=df.name, randomNoise_dim = 29, hidden_dim = 64, realData_dim =df.shape[1], lr = 0.002, num_epochs=1500, device='cpu')

AttributeError: 'DataLoader' object has no attribute 'name'

In [6]:
for i, df in enumerate(list_df):
    print(df.shape[1])
    df = prepareDataset(df, batch_size = 64, device = 'cpu', label = 'no')
    print(df2.dataset.features.size())
    xgblosses, G_losses, D_losses = vanillaGan.train(df2, prefix=df.name, randomNoise_dim = 30, hidden_dim = 64, realData_dim =df.shape[1], lr = 0.002, num_epochs=1500, device='cpu')
    print(min(xgblosses))
    plt.figure(figsize=(10,5))
    plt.title(f"{df.name}Generator and Discriminator Loss During Training")
    plt.plot(G_losses,label="G")
    plt.plot(D_losses,label="D")
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.legend()
    # plt.savefig('df_train_noTime_robust.png')
    # plt.savefig('df_train_Time_robust.png')
    # plt.savefig('df_train_noTime_standard.png')
    plt.savefig(f'GAN_{df.name}.png')
    plt.show()
    w = 10
    a = pd.DataFrame(xgblosses[:]).rolling(w).mean()
    plt.plot(a)
    plt.savefig(f'xgblosses_{i}_GAN_{df.name}.png')
    plt.show()

NameError: name 'list_df' is not defined